# Модуль 1.2 — PromptTemplate + Few‑Shot

**Цель:** научиться делать воспроизводимые промпты через шаблоны и few‑shot примеры.

**Что сделаем:**
- установим нужные библиотеки прямо в блокноте
- используем `PromptTemplate`
- добавим few‑shot примеры через `FewShotPromptTemplate`
- сравним ответы

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Шаблонный промпт (PromptTemplate)

Сделаем шаблон описания товара, чтобы убирать хардкод и получать стабильный формат.

## Разбор: почему нужен PromptTemplate

**Проблема без шаблона:** мы «вшиваем» текст прямо в код и легко ломаем формат при правках.

**Что дает PromptTemplate:**
- единый формат для всех запросов
- понятная точка изменения (меняем шаблон, а не весь код)
- удобная параметризация через `input_variables`

**Ключевые параметры:**
- `template` — строка с плейсхолдерами `{name}`
- `input_variables` — список переменных, которые обязательны для подстановки

**Практический совет:**
- делайте формат стабильным (фиксированная структура + требования к длине/стилю)
- избегайте «лишней свободы», добавляя явные ограничения (например, «2–3 предложения»)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage

base_url = os.environ.get("OPENAI_BASE_URL")
chat = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.7,
    max_tokens=256,
    base_url=base_url,
)

prompt_tmpl = PromptTemplate(
    input_variables=["product", "audience"],
    template=(
        "Сгенерируй короткое описание товара.\n"
        "Товар: {product}\n"
        "Целевая аудитория: {audience}\n"
        "Формат: 2-3 предложения, без воды."
    ),
)

filled_prompt = prompt_tmpl.format(
    product="Беспроводные наушники с активным шумоподавлением",
    audience="студенты и люди в дороге",
)

result = chat.invoke([HumanMessage(content=filled_prompt)])
print(result.content)

## Few‑Shot: учим формат на примерах

Добавим пару эталонных примеров «до/после», чтобы модель стабильно повторяла стиль.

## Разбор: Few‑Shot и его параметры

**Идея:** показать модели несколько эталонных примеров и попросить повторить стиль.

**Ключевые элементы:**
- `examples` — список словарей, каждый пример содержит вход и ожидаемый выход
- `example_prompt` — формат, в котором мы показываем пример
- `suffix` — «пустой» шаблон для нового запроса (без ответа)
- `input_variables` — переменные для `suffix`

**Как это работает:**
1. LangChain собирает строку из `example_prompt` для каждого примера.
2. Добавляет `suffix` с вашими новыми входными данными.
3. Получается единый текст, который модель продолжает в том же стиле.

**Практические рекомендации:**
- 2–5 примеров обычно достаточно; больше — дороже и не всегда лучше.
- Примеры должны быть «типичными» и отражать желаемый стиль.
- Следите за длиной примеров — они влияют на стоимость и время ответа.
- Если формат важен, закрепите его прямо в `example_prompt` (например, "Описание: ...").

## Разбор параметров ChatOpenAI

**Основные параметры, которые мы используем:**
- `model` — имя модели (например, `gpt-5.2-chat`)
- `temperature` — «креативность»; 0.2–0.7 для стабильных ответов
- `max_tokens` — ограничение длины ответа
- `base_url` — прокси URL (AITunnel)

**Практика:**
- Для обучающих примеров держите `temperature` в районе 0.5–0.7.
- Если формат важен — снижайте `temperature` до 0.2–0.4.
- `max_tokens` задавайте умеренно (256–512), чтобы не платить лишнее.

In [ ]:
from langchain.prompts import FewShotPromptTemplate

examples = [
    {
        "product": "Умная лампа",
        "audience": "люди, которые любят настраиваемое освещение",
        "output": "Умная лампа дает полный контроль над светом из телефона и поддерживает сценарии. Подходит для тех, кто хочет уют и экономию энергии.",
    },
    {
        "product": "Фитнес‑браслет",
        "audience": "новички в спорте",
        "output": "Фитнес‑браслет мягко ведет к регулярным тренировкам и показывает прогресс. Отличный старт для тех, кто только начинает.",
    },
]

example_prompt = PromptTemplate(
    input_variables=["product", "audience", "output"],
    template=(
        "Товар: {product}\n"
        "Целевая аудитория: {audience}\n"
        "Описание: {output}\n"
    ),
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix=(
        "Товар: {product}\n"
        "Целевая аудитория: {audience}\n"
        "Описание:"
    ),
    input_variables=["product", "audience"],
)

few_shot_filled = few_shot_prompt.format(
    product="Электронная книга",
    audience="люди, которые много читают в поездках",
)

few_shot_result = chat.invoke([HumanMessage(content=few_shot_filled)])
print(few_shot_result.content)